In [1]:
import pygame as pg
import sys

pygame 2.1.2 (SDL 2.0.18, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
pg.init()

(5, 0)

In [3]:
fps = 60
win_w = 800
win_h = 600

In [4]:
screen = pg.display.set_mode((win_w, win_h))
pg.display.set_caption("Hello World")

clock = pg.time.Clock()

In [5]:
class Rect:
    def __init__(self):
        self.position = (30, 90)
        self.size = (40, 40)
        self.rect = pg.Rect(self.position, self.size)
        self.colour = pg.Color("red")
        

In [6]:
rect = Rect()

In [7]:
while True:

    screen.fill((53, 43, 111))

    for event in pg.event.get():
        if event.type == pg.QUIT: 
            pg.quit()
            exit()

    clock.tick(fps) 
    pg.display.update()

    screen.blit(rect)

TypeError: argument 1 must be pygame.Surface, not Rect